In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt

# ─────────────────── Data Loading ────────────────────
df = pd.read_csv("prices.txt", sep=r"\s+", header=None)
prices = df.values.T  # shape (n_inst, T_total)

# ──────────── Fourier + ARIMA‐style Forecast Functions ────────────
def make_fourier(t, T, K):
    features = [np.ones_like(t)]
    for k in range(1, K+1):
        features.append(np.sin(2*np.pi*k*t/T))
        features.append(np.cos(2*np.pi*k*t/T))
    return np.column_stack(features)

def fourier_arima_style_forecast(prices, period, terms, focus_length, train_ratio=0.8):
    n_inst, T_total = prices.shape
    assert focus_length <= T_total, "focus_length must be ≤ total time"
    N = focus_length - 1
    train_size = int(N * train_ratio)
    n_forecast = N - train_size
    preds = np.zeros((n_inst, n_forecast))
    t = np.arange(1, focus_length)
    X_full = make_fourier(t, period, terms)
    
    for i in range(n_inst):
        series = prices[i, :focus_length]
        diff = np.diff(series)
        X_train, y_train = X_full[:train_size], diff[:train_size]
        X_test = X_full[train_size:]
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        last_price = series[train_size]
        price_pred = last_price + np.cumsum(y_pred)
        preds[i] = price_pred
    
    return preds

# ─────────── Parameters & Forecast ───────────
period       = 2 * np.pi
terms        = 1000
focus_length = 1000
train_ratio  = 0.8

predicted_prices = fourier_arima_style_forecast(
    prices,
    period,
    terms,
    focus_length,
    train_ratio
)

# ─────────── Plot Actual vs Forecast per Instrument ───────────
n_inst = prices.shape[0]
N = focus_length - 1
train_size = int(N * train_ratio)
n_forecast = predicted_prices.shape[1]

t_actual = np.arange(focus_length)               # 0 … focus_length-1
t_forecast = np.arange(train_size, train_size + n_forecast)  # aligned with diff indices

for i in range(n_inst):
    plt.figure()
    plt.plot(t_actual, prices[i, :focus_length], label="Actual")
    plt.plot(t_forecast, predicted_prices[i], label="Forecast")
    plt.title(f"Instrument {i}")
    plt.xlabel("Time")
    plt.ylabel("Price")
    plt.legend()
    plt.show()
